# ASOS Lookup Function

A user can pass either the ASOS airport code or the “city” and the script/function identifes and returns the name of the HDP station. 
E.g., “I need to find KSAC weather station” or “I need to find the Sacramento Executive Airport weather station” 

Script then identifies KSAC as “ASOSAWOS_XXXXXXXX”

Reasoning: Looking up the specific HDP stations from the attached spreadsheet was not straightforward or easy; users may very likely come to HDP knowing specifically which station they are looking for, and it’s not obvious which station is which.


In [1]:
# imports
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import os
from shapely.geometry import Point
import sys  # Used for progress bar
import datetime
import sys
import os
from io import BytesIO

plt.style.use("tableau-colorblind10")

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/all_network_stationlist_merge.csv"

## Final Function

In [2]:
def asos_station_lookup(
    code: str | None = None, city: str | None = None
) -> None:
    """
    Function that returns the HDP station ID for an input of either the
    1. four digit code or (e.g. KSAC)
    2. city (e.g. Sacramento)

    of the airport associated with the station. 

    This functon is specific to the 16 stations of interest shared by project partners.

    Paramters
    ---------
    code : str
        ASOS airport code (e.g. KSAC weather station)
    city : str
        ASOS aiport city (e.g. Sacramento Executive Airport weather station)

    Returns
    -------
    None

    """
    # Define dictionaries matching HDP station IDs to airport codes and cities
    ## this was developed beforehand matching HDP ASOSAWOS station locations with those in a list of stations provided by partners

    city_dict = {
        "Arcata": "ASOSAWOS_72594524283",
        "Eureka": "ASOSAWOS_72594524283",
        "Blue Canyon": "ASOSAWOS_72584523225",
        "Burbank": "ASOSAWOS_72288023152",
        "Fresno": "ASOSAWOS_72389093193",
        "Fullerton": "ASOSAWOS_72297603166",
        "Oakland": "ASOSAWOS_72493023230",
        "Palm Springs": "ASOSAWOS_72286893138",
        "Palmdale": "ASOSAWOS_72382023182",
        "Red Bluff": "ASOSAWOS_72591024216",
        "Riverside": "ASOSAWOS_72286903171",
        "Sacramento": "ASOSAWOS_72483023232",
        "San Diego": "ASOSAWOS_72290023188",
        "San Jose": "ASOSAWOS_72494523293",
        "Santa Maria": "ASOSAWOS_72394023273",
        "Santa Rosa": "ASOSAWOS_72495723213",
        "Torrance": "ASOSAWOS_72295503174",
    }

    code_dict = {
        "KACV": "ASOSAWOS_72594524283",
        "KBLU": "ASOSAWOS_72584523225",
        "KBUR": "ASOSAWOS_72288023152",
        "KFAT": "ASOSAWOS_72389093193",
        "KFUL": "ASOSAWOS_72297603166",
        "KOAK": "ASOSAWOS_72493023230",
        "KPSP": "ASOSAWOS_72286893138",
        "KPMD": "ASOSAWOS_72382023182",
        "KRBL": "ASOSAWOS_72591024216",
        "KRAL": "ASOSAWOS_72286903171",
        "KSAC": "ASOSAWOS_72483023232",
        "KSAN": "ASOSAWOS_72290023188",
        "KRHV": "ASOSAWOS_72494523293",
        "KSMX": "ASOSAWOS_72394023273",
        "KSTS": "ASOSAWOS_72495723213",
        "KTOA": "ASOSAWOS_72295503174",
    }

    # If user inputs an airport code
    if code:
        if code in code_dict:
            hdp_station = code_dict[code]
            print(f"The HDP station name for input airport code {code} is {hdp_station}")
        else:
            print(f"Input code '{code}' not in station dictionary.")
    # If user inputs a city
    elif city:
        # this catches cases in which the user inputs the entire airport name
        hdp_station = [val for key, val in city_dict.items() if key in city]
        if hdp_station:
            # now pull the ID out from the list that is returned above
            hdp_station = hdp_station[0]
            print(f"The HDP station name for input airport city '{city}' is {hdp_station}")
        else: 
            print(f"Input city '{city}' not in station dictionary.")
    else:
        print("Please input either an airport code (e.g. code='KSAC') or city (e.g. city='Sacramento').")

    return None

In [13]:
asos_station_lookup(code=None, city="Sacramento Executive Airport")

The HDP station name for input airport city 'Sacramento Executive Airport' is ASOSAWOS_72483023232


In [ ]:
# test invalid inputs
asos_station_lookup(code='test', city="Sacramento Executive Airport")

Input code 'test' not in station dictionary.


## Development

grab the station codes from the pull station list version and append as a new column into the merge station list.

In [2]:
# load merge station list
merge_list = pd.read_csv(stations_csv_path)
merge_list = merge_list[merge_list["network"] == "ASOSAWOS"]

In [3]:
# load pull station list
pull_list = pd.read_csv(
    "s3://wecc-historical-wx/1_raw_wx/all_network_stationlist_pull.csv"
)
pull_list = pull_list[pull_list["network"] == "ASOSAWOS"]

In [4]:
pull_list = pull_list[['latitude','longitude','name']]

In [5]:
len(merge_list)

455

In [6]:
both_list = pd.merge(merge_list,pull_list, on=['latitude','longitude'],how='left')
both_list

,Unnamed: 0,era-id,latitude,longitude,elevation,start-date,end-date,pulled,time_checked,network,...,hurs_nobs,sfcwind_nobs,sfcwind_dir_nobs,rsds_nobs,total_nobs,qaqc,time_qaqc,merged,time_merge,name
0,0,ASOSAWOS_72681024131,43.567,-116.241,860.4504,1931-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,444433,379116,87216,470754,Y,2025-05-01 19:48:20+00:00,Y,2025-06-12 21:37:35+00:00,BOISE AIR TERMINAL/GOWEN FD AIRPORT
1,1,ASOSAWOS_72274803914,32.950,-111.767,-30479.6952,2006-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,318667,209160,0,326820,Y,2025-05-01 19:39:45+00:00,Y,2025-06-12 18:29:49+00:00,CASA GRANDE MUNICIPAL ARPT
2,2,ASOSAWOS_72665494054,44.381,-106.720,1503.8832,2006-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,189465,168125,0,198450,Y,2025-05-01 19:40:34+00:00,Y,2025-06-12 20:57:53+00:00,JOHNSON COUNTY AIRPORT
3,3,ASOSAWOS_72278303185,33.466,-111.721,423.3672,2006-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,88523,75118,0,91095,Y,2025-05-01 19:36:44+00:00,Y,2025-06-12 18:28:57+00:00,FALCON FIELD AIRPORT
4,4,ASOSAWOS_72274953128,33.269,-111.813,378.8664,2006-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,91799,65071,0,92525,Y,2025-05-01 19:35:04+00:00,Y,2025-06-12 18:27:52+00:00,CHANDLER MUNICIPAL AIRPORT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,452,ASOSAWOS_72679624138,45.258,-112.554,NaN,1973-01-01 00:00:00+00:00,1997-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,162295,138191,0,179969,Y,2025-05-20 17:17:47+00:00,Y,2025-06-12 20:59:59+00:00,DILLON
490,452,ASOSAWOS_72679624138,45.258,-112.554,NaN,1973-01-01 00:00:00+00:00,1997-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,162295,138191,0,179969,Y,2025-05-20 17:17:47+00:00,Y,2025-06-12 20:59:59+00:00,DILLON AIRPORT
491,453,ASOSAWOS_72667524037,46.427,-105.883,NaN,1973-01-01 00:00:00+00:00,1983-05-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,35748,33033,0,35776,Y,2025-05-01 19:38:35+00:00,Y,2025-06-12 18:35:37+00:00,MILES CITY MUNICIPAL ARPT
492,454,ASOSAWOS_72475723176,38.417,-113.017,NaN,1977-01-01 00:00:00+00:00,1983-04-30 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,8853,8454,0,28545,Y,2025-05-20 16:29:26+00:00,N,NaN,MILFORD MUNICIPAL AP


In [21]:
len(both_list)

494

Need to determine why the final dataframe is longer than both inputs.
Onward to extracting city and airport code.

In [7]:
len(pull_list)

446

In [8]:
len(pull_list['name'].unique())
# there are 439 unique airports, which some repeats

439

In [9]:
airport_list = pd.read_csv(
    "../../skillshare_demo/Airport_Boundaries.csv"
)

In [10]:
airport_list = airport_list[['FACILITY','CITY','AIRPORTID']]

In [11]:
airport_list

,FACILITY,CITY,AIRPORTID
0,STOCKTON METROPOLITAN AIRPORT,Stockton,SCK
1,BERMUDA DUNES EXECUTIVE AIRPORT,Palm Springs,UDD
2,BAKER AIRPORT,Baker,0O2
3,CORNING MUNICIPAL AIRPORT,Corning,0O4
4,JACQUELINE COCHRAN REGIONAL AIRPORT,Thermal,TRM
...,...,...,...
216,"SAN JOSE INTERNATIONAL AIRPORT, NORMAN Y. MINETA",San Jose,SJC
217,SAN FRANCISCO INTERNATIONAL AIRPORT,San Francisco,SFO
218,METROPOLITAN OAKLAND INTERNATIONAL AIRPORT,Oakland,OAK
219,STOCKTON METROPOLITAN AIRPORT,Stockton,SCK


In [12]:
airport_list[airport_list["FACILITY"] == "PALM SPRINGS INTL AIRPORT"]

,FACILITY,CITY,AIRPORTID


In [13]:
airport_list['FACILITY'].unique()

array(['STOCKTON METROPOLITAN AIRPORT', 'BERMUDA DUNES EXECUTIVE AIRPORT',
       'BAKER AIRPORT', 'CORNING MUNICIPAL AIRPORT',
       'JACQUELINE COCHRAN REGIONAL AIRPORT', 'FRAZIER LAKE AIRPARK',
       'LAKE TAHOE AIRPORT', 'LAMPSON FIELD AIRPORT', 'LODI AIRPORT',
       'MONTAGUE-YREKA, ROHRER FIELD', 'NUT TREE AIRPORT',
       'SELMA AIRPORT', 'SHELTER COVE AIRPORT', 'SONOMA SKYPARK AIRPORT',
       'SONOMA VALLEY AIRPORT', 'TEHACHAPI MUNICIPAL AIRPORT',
       'TRUCKEE-TAHOE AIRPORT', 'TWENTYNINE PALMS AIRPORT',
       'UKIAH MUNICIPAL AIRPORT', 'VAN NUYS AIRPORT',
       'WARD FIELD AIRPORT', 'WATSONVILLE MUNICIPAL AIRPORT',
       'WHITEMAN AIRPORT', 'GENERAL WILLIAM M FOX AIRPORT',
       'WILLOWS - GLENN COUNTY AIRPORT', 'ZAMPERINI FIELD AIRPORT',
       'CALIFORNIA PINES AIRPORT',
       'CHARLES M. SCHULZ, SONOMA COUNTY AIRPORT',
       'CHEMEHUEVI VALLEY AIRPORT', 'CHOWCHILLA AIRPORT',
       'ECKERT FIELD AIRPORT', 'GANSNER AIRPORT',
       'GRAVELLY VALLEY AIRPORT', 'GUS

In [14]:
airport_merge = pd.merge(
    both_list, airport_list, left_on=["name"],right_on=['FACILITY'], how="left"
)

In [15]:
len(airport_merge)
# LOL two more rows added after this merge

496

In [16]:
airport_merge['name'].unique()

array(['BOISE AIR TERMINAL/GOWEN FD AIRPORT',
       'CASA GRANDE MUNICIPAL ARPT', 'JOHNSON COUNTY AIRPORT',
       'FALCON FIELD AIRPORT', 'CHANDLER MUNICIPAL AIRPORT',
       'GILLETTE-CAMPBELL CO. ARPT', 'CHAMA AWRS', 'WINDOW ROCK AIRPORT',
       'YUBA COUNTRY AIRPORT', 'LAKE COUNTY AIRPORT',
       'LEMHI COUNTY AIRPORT', 'MONUMENT HILL AWOS-3 ARPT',
       'SAFFORD REGIONAL AIRPORT', 'PAGE MUNICIPAL AIRPORT',
       'TACOMA NARROWS AIRPORT', 'ARLINGTON MUNICIPAL ARPT',
       'FRIDAY HARBOR AIRPORT', 'NORTH AUXILIARY AIRFIELD EDWARDS AFB',
       'LA VETA PASS AWOS-3 ARPT', 'FULLERTON MUNICIPAL ARPT',
       'VERNAL AIRPORT', 'SIDNEY-RICHLAND MUNI ARPT',
       'MADERA MUNICIPAL AIRPORT', 'L M CLAYTON AIRPORT',
       'MONTROSE REGIONAL AIRPORT', 'GRAND CANYON NATL PARK APT',
       'TELLURIDE REGIONAL AIRPORT', 'LAMAR MUNICIPAL AIRPORT',
       'SOUTH BIG HORN COUNTY ARPT', 'ARTESIA MUNICIPAL AIRPORT',
       'NOGALES INTERNATIONAL ARPT', 'DAWSON COMMUNITY AIRPORT',
       'NORT

In [17]:
airport_list[airport_list["FACILITY"] == "SAN FRANCISCO INTERNATIONAL AIRPORT"]

,FACILITY,CITY,AIRPORTID
207,SAN FRANCISCO INTERNATIONAL AIRPORT,San Francisco,SFO
217,SAN FRANCISCO INTERNATIONAL AIRPORT,San Francisco,SFO


In [18]:
both_list[both_list["name"] == "SAN FRANCISCO INTERNATIONAL AIRPORT"]

,Unnamed: 0,era-id,latitude,longitude,elevation,start-date,end-date,pulled,time_checked,network,...,hurs_nobs,sfcwind_nobs,sfcwind_dir_nobs,rsds_nobs,total_nobs,qaqc,time_qaqc,merged,time_merge,name
282,266,ASOSAWOS_72494023234,37.62,-122.366,3.048,1973-01-01 00:00:00+00:00,2022-12-31 00:00:00+00:00,Y,2023-01-09 16:28:28+00:00,ASOSAWOS,...,0,452605,410094,0,464741,Y,2025-05-01 19:44:47+00:00,Y,2025-06-12 21:04:47+00:00,SAN FRANCISCO INTERNATIONAL AIRPORT


In [19]:
names_list = both_list['name']

I don't think that's going to work. Let's get the city and codes another way?

In [20]:
(firstWord, rest) = names_list[0].split(maxsplit=1)

In [21]:
city_list = []
#city_list += [i.split(maxsplit=1) for i in names_list]
city_list += [i.split()[0] for i in names_list]

In [22]:
unique_names = list(set(city_list))

In [23]:
len(unique_names)

362

## Scraps

I matched up ASOSAWOS stations from the merge station list with the lat lons of the stations in the list that Mithra shared.

In [ ]:
# load station list
station_list = pd.read_csv(stations_csv_path)
station_list = station_list[station_list['network']=='ASOSAWOS']

In [45]:
# convert to geodataframe
asos_stns = gpd.GeoDataFrame(
    station_list,
    geometry=gpd.points_from_xy(
        station_list.longitude, station_list.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# load in list that Mithra shared (stored in separate repo)
lookup_table = pd.read_csv("../../skillshare_demo/lookup_table.csv")

In [ ]:
# convert it to a geodataframe
lookup_table = gpd.GeoDataFrame(
    lookup_table,
    geometry=gpd.points_from_xy(
        lookup_table.longitude, lookup_table.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# and filter for ease of use later
lookup_table = lookup_table[
    ["Airport Code", "City", "latitude", "longitude",'geometry']
]

In [ ]:
# establish buffers around the point geometries, as they will not be exact matches

# define the error margin (buffer distance)
error_margin = 0.1  # Adjust this value as needed

# apply buffer to account for the error margin
lookup_table_buffered = lookup_table.copy()
lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)

asos_stns_buffered = asos_stns.copy()
asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)

/tmp/ipykernel_31410/1684058703.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)
/tmp/ipykernel_31410/1684058703.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)


In [ ]:
# perform the overlay operation (e.g., intersection)
matchup_buffered = gpd.overlay(asos_stns_buffered, lookup_table_buffered, how="intersection")

In [ ]:
# clean things up 
matchup_buffered = matchup_buffered[["era-id", "longitude_1", "latitude_1","longitude_2", "latitude_2",'Airport Code','City']]

Now go through each city in the list (which I added manually before starting this process) and check which HDP station is a match. I added the station ID to a copy of the list in my drive along the way.

In [102]:
matchup_buffered[matchup_buffered["City"] == "San Jose"]

,era-id,longitude_1,latitude_1,longitude_2,latitude_2,Airport Code,City
19,ASOSAWOS_72494523293,-121.924,37.359,-121.82,37.33,KRHV,San Jose


I then downloaded a cleaned version of the table and load it in here (again, stored in a separate repo).

In [ ]:
matchup_table = pd.read_csv("../../skillshare_demo/final_lookup_table.csv")

And generated two dictionaries from it - one for airport codes, another for airport cities

In [123]:
code_dict = pd.Series(matchup_table["HDP station"].values, index=matchup_table["Airport Code"]).to_dict()

In [124]:
city_dict = pd.Series(
    matchup_table["HDP station"].values, index=matchup_table["City"]
).to_dict()

The dictionaries are generated manually in the function itself.